In [0]:
# reset_environment.ipynb
# Reset Spark/Delta state without touching ADLS raw.
#
# - Stops all active Structured Streaming queries
# - Drops all kardia_* databases
# - Deletes DBFS /kardia/* (checkpoints, schemas, bronze/silver/gold, bad records, etc.)
# - Leaves ADLS raw container intact

from kflow.config import ensure_adls_auth
import re

# 1) Stop all active Structured Streaming queries
for stream in spark.streams.active:
    print(f"Stopping stream: {stream.id}")
    stream.stop()
    stream.awaitTermination(timeout=30)

# 2) Drop all user-created databases prefixed with 'kardia_'
for db in spark.catalog.listDatabases():
    if db.name.startswith("kardia_"):
        print(f"Dropping database: {db.name}")
        spark.sql(f"DROP DATABASE {db.name} CASCADE")

# 3) Delete DBFS subtrees for pipeline state
dbfs_dirs = [
    "dbfs:/kardia/bronze",
    "dbfs:/kardia/silver",
    "dbfs:/kardia/gold",
    "dbfs:/kardia/checkpoints",
    "dbfs:/kardia/schemas",
    "dbfs:/kardia/bad",
]

for path in dbfs_dirs:
    try:
        dbutils.fs.rm(path, recurse=True)
        print(f"Deleted DBFS path: {path}")
    except Exception as e:
        print(f"DBFS cleanup warning for {path}: {e}")

print("Kardia environment reset complete (ADLS raw untouched).")